# Context is King, JinaAI vs Tavily vs AskNews



In [ ]:
pip install asknews tavily-python langchain langchain_openai ipykernel ragas python-rapidjson requests==2.31.0 exa-py tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Note: we use the google colab secrets in the left hand panel. Please add your secrets in order to run the notebook.

In [ ]:
import requests
import time
from asknews_sdk import AskNewsSDK
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import rapidjson
from tavily import TavilyClient
import os
from IPython.display import Markdown, display
from datasets import Dataset
from ragas.metrics import context_precision, answer_correctness
from ragas import evaluate
from google.colab import userdata
from collections import defaultdict
from datetime import datetime, timezone
from exa_py import Exa
import tiktoken
import time


# Head over to each of the services to get your credentials
# - [JinaAI](https://jina.ai/reader/#apiform)
# - [Tavily](https://tavily.com/)
# - [AskNews](https://my.asknews.app/plans)
# - [Exa](https://exa.ai)

# Make sure to add your secrets in the "Secrets" tab of the
# Google Colab notebook

model = "gpt-3.5-turbo-16k" #"gpt-4o"

tavily = TavilyClient(api_key=userdata.get('TAVILY_KEY'))
jina_apikey = userdata.get('JINAAI_KEY')

def call_jina(query):
    url = f"https://s.jina.ai/{query}"
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {jina_apikey}"
    }

    response = requests.get(url, headers=headers)

    return response.json()

ask = AskNewsSDK(
    client_id=userdata.get('ASKNEWS_CLIENT_ID'),
    client_secret=userdata.get('ASKNEWS_SECRET'),
    scopes=["news"]
)

llm = ChatOpenAI(
    model=model,
    temperature=0.0,
    api_key=userdata.get('OPENAI_KEY'),
    streaming=False,
)
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

exa = Exa(api_key=userdata.get('EXA_KEY'))



We build a set of queries to test broad and niche topics alike.

Please note: these questions are all time-sensitive, and as such, the ground truth will change. In some cases, like latest_news it will change by the hour. In other cases, it may take longer (hopefully much longer for the Greenland ice sheet thickness question). If you re-run the google colab, you will need to update some of these ground truths accordingly.

In [ ]:
queries = [
    {
        "type": "latest news 1",
        "query": "What was the score of the Yankees game?",
        "ground_truth": "The New York Yankees beat Kansas City Royals, with the final score being 10-1 in favor of the Yankees."
    },
    {
        "type": "latest news 2",
        "query": "What is the current price of Bitcoin?",
        "ground_truth": "The current price of bitcoin is $69,562.00."
    },
    {
        "type": "general web 1",
        "query": "Who won the SuperBowl this year?",
        "ground_truth": "The Kansas City Chiefs won the SuperBowl."
    },
    {
        "type": "general web 2",
        "query": "Who is Tom Cruise married to right now?",
        "ground_truth": "Tom Cruise is not currently married."
    },
    {
        "type": "knowledge search 1",
        "query": "What are the highlights of the latest Node.js version?",
        "ground_truth": "Highlights include require()-ing ESM graphs, WebSocket client, updates of the V8 JavaScript engine"
    },
    {
        "type": "knowledge search 2",
        "query": "What is the ice sheet area and thickness of Greenland?",
        "ground_truth": "Greenland's ice sheet has an area of 1.7 million square kilometers and an average thickness of 2.3 kilometers (1.4 miles)."
    },
    {
        "type": "google-esque 1",
        "query": "What is the current Fed interest rate?",
        "ground_truth": "The Federal Reserve's current interest is between 5.25% and 5.50%."
    },
    {
        "type": "google-esque 2",
        "query": "What's the cost of living in Amsterdam?",
        "ground_truth": "A family of four's estimated monthly costs are €5,632, while a single person's estimated monthly costs: €3,317."
    },
]

# make a directory for storing files
dir_name = "comparison_files"
os.makedirs(dir_name, exist_ok=True)

JinaAI and Tavily need a bit of massaging to build something interpretable by the LLM, we add those functions here.

In [ ]:
def build_tavily_context(tavily_response):
    context = ""
    contexts = []
    for i, obj in enumerate(tavily_response["results"]):
        text = f"<doc>\n[{i+1}]\n{obj['title']}\n{obj['content']}\n</doc>\n\n"
        context += text
        contexts.append(text)
    return context, contexts


def build_jina_context(jina_response):
    context = ""
    contexts = []
    if "data" not in jina_response or jina_response["data"] == None:
        return context, contexts
    for i, article in enumerate(jina_response["data"]):
        text = f"<doc>\n[{i+1}]\n{article['title']}\n{article['description']}\n</doc>\n\n"
        context += text
        contexts.append(text)
    return context, contexts

def build_asknews_context(input_string):
  docs_list = input_string.strip().split('</doc>')
  docs_list = [doc.strip() + '</doc>' for doc in docs_list if doc.strip()]
  return docs_list

def build_exa_context(exa_response):
  context = ""
  contexts = []
  for i, article in enumerate(exa_response.results):
    text = f"<doc>\n[{i+1}]\nTitle:{article.title}\nHighlights:{article.highlights[0]}\nPublished:{article.published_date}\n</doc>\n\n"
    context += text
    contexts.append(text)
  return context, contexts

def print_md(string):
    display(Markdown(string))

In [ ]:
def get_llm_response(query, news_context):

    current_time = datetime.now(tz=timezone.utc).strftime("%B %d %Y %H:%M")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "A conversation between a helpful Assistant and a User. The User asks the Assistant a question and gives the Assistant a set of documents that might help the Assistant to respond. The Assistant aims to answers the User question as concisely as possible.",
            ),
            ("human",
             f"Given the current time is {current_time} and the following documents:\n\n----START DOCUMENT CONTEXT----{{news_context}}-----END DOCUMENT CONTEXT----\n\nPlease answer my question:\n\n{{question}}"),
        ]
    )


    complete = False
    while not complete:
      try:
        # openai throws "Sorry! Repetitive prompt, try a different prompt"...trying to instantiate a new instance each time perhaps.
        llm = ChatOpenAI(
            model=model,
            temperature=0.0,
            api_key=userdata.get('OPENAI_KEY'),
            streaming=False,
        )
        agent = prompt | llm
        response = agent.invoke({
            "question": query,
            "news_context": news_context
        })
        complete = True
      except Exception as e:
        print(f"Failed with {e}, trying again")
        time.sleep(30)
        continue

    text = prompt.invoke({
            "question": query,
            "news_context": news_context
        })
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-16k")
    input_tokens = len(encoding.encode(str(text)))

    return response.content, input_tokens


In [ ]:
def get_answer_correctness(query, contexts, answer, ground_truth):

  data_sample = {
      "question": [query],
      "contexts": [contexts],
      "answer": [answer],
      "ground_truth": [ground_truth]
  }
  dataset = Dataset.from_dict(data_sample)
  complete = False
  while not complete:
    # OpenAI sometimes complains and says "Sorry! Repetitive prompt, try a different prompt"...trying to instantiate a new instance each time perhaps.
    try:
      llm = ChatOpenAI(
          model=model,
          temperature=0.0,
          api_key=userdata.get('OPENAI_KEY'),
          streaming=False,
      )
      score = evaluate(dataset,metrics=[answer_correctness, context_precision], llm=llm)
      complete = True
    except Exception as e:
        print(f"Failed with {e}, trying again")
        time.sleep(30)
        continue

  return score['answer_correctness'], score['context_precision']


In [ ]:
from datetime import datetime, timedelta

scores = {
    'asknews_scores':
  {'answer_correctness': defaultdict(list), 'context_precision': defaultdict(list), 'time': defaultdict(list), 'tokens': defaultdict(list)  },
    'jina_scores':
  {'answer_correctness': defaultdict(list), 'context_precision': defaultdict(list), 'time': defaultdict(list), 'tokens': defaultdict(list)  },
    'tavily_scores':
  {'answer_correctness': defaultdict(list), 'context_precision': defaultdict(list), 'time': defaultdict(list), 'tokens': defaultdict(list)  },
    'exa_scores':
  {'answer_correctness': defaultdict(list), 'context_precision': defaultdict(list), 'time': defaultdict(list), 'tokens': defaultdict(list) },
}

trials = 3

for trial in range(trials):
  for query in queries:
      query_str = query["query"]
      query_type = query["type"]
      gt = query["ground_truth"]
      query_hash = str(hash(query_str))

      # Exa
      if "latest_news" in query_type:
        one_week_ago = (datetime.now() - timedelta(days=1))
        date_cutoff = one_week_ago.strftime("%Y-%m-%d")
      else:
        date_cutoff = None
      start = time.time()
      exa_response = exa.search_and_contents(
        query_str,
        use_autoprompt=True,
        num_results=5,
        start_published_date=date_cutoff,
        highlights=True
      )
      scores['exa_scores']['time'][query_type].append(time.time() - start)

      exa_context, contexts = build_exa_context(exa_response)
      exa_llm, input_tokens = get_llm_response(query_str, exa_context)

      exa_score, precision = get_answer_correctness(query_str, contexts, exa_llm, gt)
      scores['exa_scores']['answer_correctness'][query_type].append(exa_score)
      scores['exa_scores']['context_precision'][query_type].append(precision)
      scores['exa_scores']['tokens'][query_type].append(input_tokens)

      # AskNews
      start = time.time()
      asknews_context = ask.news.search_news(
          query_str,
          return_type="string",
          n_articles=5,
          strategy="latest news" if "latest news" in query_type else "news knowledge"
      ).as_string
      scores['asknews_scores']['time'][query_type].append(time.time() - start)

      contexts = build_asknews_context(asknews_context)
      asknews_llm, input_tokens = get_llm_response(query_str, asknews_context)

      asknews_score, precision = get_answer_correctness(query_str, contexts, asknews_llm, gt)
      scores['asknews_scores']['answer_correctness'][query_type].append(asknews_score)
      scores['asknews_scores']['context_precision'][query_type].append(precision)
      scores['asknews_scores']['tokens'][query_type].append(input_tokens)


      # JinaAI
      start = time.time()
      # Jina many times gives empty responses...really helpful. need to use a while loop
      completed = False
      while not completed:
        jina_response = call_jina(query_str)
        if "data" not in jina_response or not jina_response["data"]:
          print(f"Jina returned empty context for {query_str}, trying again")
        if contexts:
          completed = True
          break


      scores['jina_scores']['time'][query_type].append(time.time() - start)
      jina_context, contexts = build_jina_context(jina_response)

      jina_llm, input_tokens = get_llm_response(query_str, jina_context)

      jina_score, precision = get_answer_correctness(query_str, contexts, jina_llm, gt)
      scores['jina_scores']['answer_correctness'][query_type].append(jina_score)
      scores['jina_scores']['context_precision'][query_type].append(precision)
      scores['jina_scores']['tokens'][query_type].append(input_tokens)

      # Tavily
      # For advanced search:
      start = time.time()
      response = tavily.search(query=query_str, search_depth="advanced")
      scores['tavily_scores']['time'][query_type].append(time.time() - start)

      tavily_context, contexts = build_tavily_context(response)
      tavily_llm, input_tokens = get_llm_response(query_str, tavily_context)

      tavily_score, precision = get_answer_correctness(query_str, contexts, tavily_llm, gt)
      scores['tavily_scores']['answer_correctness'][query_type].append(tavily_score)
      scores['tavily_scores']['context_precision'][query_type].append(precision)
      scores['tavily_scores']['tokens'][query_type].append(input_tokens)

      # print the results
      print_md(f"## {query_str} - {query_type} - TRIAL {trial}")
      print_md(f"### Exa")
      print_md("#### Ragas Correctness/Precision")
      print_md(f"{scores['exa_scores']['answer_correctness'][query_type][-1]}/{scores['exa_scores']['context_precision'][query_type][-1]}")
      print_md("#### LLM Response")
      print_md(f"Input Tokens: {scores['exa_scores']['tokens'][query_type][-1]}")
      print_md(exa_llm)
      print_md(f"#### Context")
      print(exa_context)
      print_md(f"### AskNews")
      print_md("#### Ragas Correctness/Precision")
      print_md(f"{scores['asknews_scores']['answer_correctness'][query_type][-1]}/{scores['asknews_scores']['context_precision'][query_type][-1]}")
      print_md("#### LLM Response")
      print_md(f"Input Tokens: {scores['asknews_scores']['tokens'][query_type][-1]}")
      print_md(asknews_llm)
      print_md(f"#### Context")
      print(asknews_context)
      print_md(f"### JinaAI")
      print_md("#### Ragas Correctness/Precision")
      print_md(f"{scores['jina_scores']['answer_correctness'][query_type][-1]}/{scores['jina_scores']['context_precision'][query_type][-1]}")
      print_md("#### LLM Response")
      print_md(f"Input Tokens: {scores['jina_scores']['tokens'][query_type][-1]}")
      print_md(jina_llm)
      print_md(f"#### Context")
      print(jina_context)
      print_md(f"### Tavily")
      print_md("#### Ragas Correctness/Precision")
      print_md(f"{scores['tavily_scores']['answer_correctness'][query_type][-1]}/{scores['tavily_scores']['context_precision'][query_type][-1]}")
      print_md(f"#### LLM Response")
      print_md(f"Input Tokens: {scores['tavily_scores']['tokens'][query_type][-1]}")
      print_md(tavily_llm)
      print_md(f"#### Context")
      print(tavily_context)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

## What was the score of the Yankees game? - latest news 1 - TRIAL 0

...

#### LLM Response

Input Tokens: 997

The cost of living in Amsterdam varies depending on the source. According to the documents provided, the average monthly cost of living for a single person ranges from €1,148.3 to €3,317. For a family of four, the estimated monthly costs range from €4,012.9 to €5,632.

#### Context

<doc>
[1]
Amsterdam: Cost of Living, Salaries, Prices for Rent & food
The average cost of living in Amsterdam is $2545, which is in the top 7% of the most expensive cities in the world, ranked 612th out of 9294 in our global list and 1st out of 70 in the Netherlands.. The median after-tax salary is $3788, which is enough to cover living expenses for 1.5 months.Ranked 37th (TOP 0.4%) in the list of best places to live in the world and 1st best city to live in the ...
</doc>

<doc>
[2]
The cost of living in Amsterdam: all you need to know in 2024 - DutchReview
But buying a house and getting a Dutch mortgage is a whole separate story, so let’s stick with asking prices here:
To buy a simple one-bedroom apartment on the low end of the spectrum, on the outskirts of Amsterdam, you might be able to get away with paying around €200,000.
READ MORE | Top tips for buying a house in the Netherlands
For a higher-end three-bedroom apartment in the centre, you can expect to spend something closer to €

In [ ]:
print(f"Total scores: {scores}")
current_time = int(datetime.now().timestamp())
with open(f"{dir_name}/results_{current_time}.json", "w") as f:
    f.write(rapidjson.dumps(scores))

Total scores: {'asknews_scores': {'answer_correctness': defaultdict(<class 'list'>, {'latest news 1': [0.5326700826216876, 0.5326816268933923, 0.5326717266963672], 'latest news 2': [0.23458329281390514, 0.6084666566871479, 0.21260630998865776], 'general web 1': [0.9964321037279145, 0.9964321037279145, 0.9964321037279145], 'general web 2': [0.7274277297329907, 0.7274277297329907, 0.7274277297329907], 'knowledge search 1': [0.8585659273515399, 0.7270271373278524, 0.7270271373278524], 'knowledge search 2': [0.2278383009268854, 0.21129021228175204, 0.21129021228175204], 'google-esque 1': [0.9954673044093185, 0.9954673044093185, 0.9954673044093185], 'google-esque 2': [0.20345628792161424, 0.5366634756425607, 0.5366634756425607]}), 'context_precision': defaultdict(<class 'list'>, {'latest news 1': [0.999999999975, 0.6791666666496875, 0.6791666666496875], 'latest news 2': [0.499999999975, 0.499999999975, 0.5333333333155555], 'general web 1': [0.94999999997625, 0.94999999997625, 0.949999999976

Now we build the latex table (thanks to GPT4o)

In [ ]:
# Extract the data
data = scores

# Helper function to format correctness, time, and token values with colors
def format_value(value, is_correctness_or_precision, is_max, is_min):
    if is_correctness_or_precision:
        if is_max:
            return f"\\textcolor{{darkgreen}}{{{value:.2f}}}"
        elif is_min:
            return f"\\textcolor{{red}}{{{value:.2f}}}"
    else:
        if is_min:
            return f"\\textcolor{{darkgreen}}{{{value:.2f}}}"
        elif is_max:
            return f"\\textcolor{{red}}{{{value:.2f}}}"
    return f"{value:.2f}"

# Initialize the LaTeX table
latex_table = r"""
\begin{table}[ht]
\centering
\makebox[\textwidth][c]{
\begin{tabular}{lcccc|cccc}
\toprule
 \textbf{Query type} & \multicolumn{4}{c}{\textbf{AskNews}} & \multicolumn{4}{c}{\textbf{JinaAI}} \\ \cline{2-9}
 & Correctness & Context Precision & Time (s) & Tokens & Correctness & Context Precision & Time (s) & Tokens \\ \hline
"""

# Define the query types and initialize lists for average calculations
query_types = [q["type"] for q in queries]

asknews_scores = data["asknews_scores"]["answer_correctness"]
jina_scores = data["jina_scores"]["answer_correctness"]
tavily_scores = data["tavily_scores"]["answer_correctness"]
exa_scores = data["exa_scores"]["answer_correctness"]

asknews_precision = data["asknews_scores"]["context_precision"]
jina_precision = data["jina_scores"]["context_precision"]
tavily_precision = data["tavily_scores"]["context_precision"]
exa_precision = data["exa_scores"]["context_precision"]

asknews_times = data["asknews_scores"]["time"]
jina_times = data["jina_scores"]["time"]
tavily_times = data["tavily_scores"]["time"]
exa_times = data["exa_scores"]["time"]

asknews_tokens = data["asknews_scores"]["tokens"]
jina_tokens = data["jina_scores"]["tokens"]
tavily_tokens = data["tavily_scores"]["tokens"]
exa_tokens = data["exa_scores"]["tokens"]

# Compute and format the average row
avg_correctness = [
    sum([sum(asknews_scores[q]) / len(asknews_scores[q]) for q in query_types]) / len(query_types),
    sum([sum(jina_scores[q]) / len(jina_scores[q]) for q in query_types]) / len(query_types),
    sum([sum(tavily_scores[q]) / len(tavily_scores[q]) for q in query_types]) / len(query_types),
    sum([sum(exa_scores[q]) / len(exa_scores[q]) for q in query_types]) / len(query_types)
]

avg_precision = [
    sum([sum(asknews_precision[q]) / len(asknews_precision[q]) for q in query_types]) / len(query_types),
    sum([sum(jina_precision[q]) / len(jina_precision[q]) for q in query_types]) / len(query_types),
    sum([sum(tavily_precision[q]) / len(tavily_precision[q]) for q in query_types]) / len(query_types),
    sum([sum(exa_precision[q]) / len(exa_precision[q]) for q in query_types]) / len(query_types)
]

avg_time = [
    sum([sum(asknews_times[q]) / len(asknews_times[q]) for q in query_types]) / len(query_types),
    sum([sum(jina_times[q]) / len(jina_times[q]) for q in query_types]) / len(query_types),
    sum([sum(tavily_times[q]) / len(tavily_times[q]) for q in query_types]) / len(query_types),
    sum([sum(exa_times[q]) / len(exa_times[q]) for q in query_types]) / len(query_types)
]

avg_tokens = [
    sum([sum(asknews_tokens[q]) / len(asknews_tokens[q]) for q in query_types]) / len(query_types),
    sum([sum(jina_tokens[q]) / len(jina_tokens[q]) for q in query_types]) / len(query_types),
    sum([sum(tavily_tokens[q]) / len(tavily_tokens[q]) for q in query_types]) / len(query_types),
    sum([sum(exa_tokens[q]) / len(exa_tokens[q]) for q in query_types]) / len(query_types)
]

max_avg_correctness = max(avg_correctness)
min_avg_correctness = min(avg_correctness)
max_avg_precision = max(avg_precision)
min_avg_precision = min(avg_precision)
min_avg_time = min(avg_time)
max_avg_time = max(avg_time)
max_avg_tokens = max(avg_tokens)
min_avg_tokens = min(avg_tokens)

# Function to compute max and min across all models for each query type
def get_max_min_values_for_query(query):
    correctness_values = [
        sum(asknews_scores[query]) / len(asknews_scores[query]),
        sum(jina_scores[query]) / len(jina_scores[query]),
        sum(tavily_scores[query]) / len(tavily_scores[query]),
        sum(exa_scores[query]) / len(exa_scores[query])
    ]

    precision_values = [
        sum(asknews_precision[query]) / len(asknews_precision[query]),
        sum(jina_precision[query]) / len(jina_precision[query]),
        sum(tavily_precision[query]) / len(tavily_precision[query]),
        sum(exa_precision[query]) / len(exa_precision[query])
    ]

    time_values = [
        sum(asknews_times[query]) / len(asknews_times[query]),
        sum(jina_times[query]) / len(jina_times[query]),
        sum(tavily_times[query]) / len(tavily_times[query]),
        sum(exa_times[query]) / len(exa_times[query])
    ]

    token_values = [
        sum(asknews_tokens[query]) / len(asknews_tokens[query]),
        sum(jina_tokens[query]) / len(jina_tokens[query]),
        sum(tavily_tokens[query]) / len(tavily_tokens[query]),
        sum(exa_tokens[query]) / len(exa_tokens[query])
    ]

    return {
        "max_correctness": max(correctness_values),
        "min_correctness": min(correctness_values),
        "max_precision": max(precision_values),
        "min_precision": min(precision_values),
        "min_time": min(time_values),
        "max_time": max(time_values),
        "max_tokens": max(token_values),
        "min_tokens": min(token_values)
    }

# Generate the top table rows
for query in query_types:
    values = get_max_min_values_for_query(query)

    correctness_values = [
        sum(asknews_scores[query]) / len(asknews_scores[query]),
        sum(jina_scores[query]) / len(jina_scores[query]),
    ]

    precision_values = [
        sum(asknews_precision[query]) / len(asknews_precision[query]),
        sum(jina_precision[query]) / len(jina_precision[query]),
    ]

    time_values = [
        sum(asknews_times[query]) / len(asknews_times[query]),
        sum(jina_times[query]) / len(jina_times[query]),
    ]

    token_values = [
        sum(asknews_tokens[query]) / len(asknews_tokens[query]),
        sum(jina_tokens[query]) / len(jina_tokens[query]),
    ]

    latex_table += f"{query.capitalize().replace('_', ' ')} & " + \
        f"{format_value(correctness_values[0], True, correctness_values[0] == values['max_correctness'], correctness_values[0] == values['min_correctness'])} & " + \
        f"{format_value(precision_values[0], True, precision_values[0] == values['max_precision'], precision_values[0] == values['min_precision'])} & " + \
        f"{format_value(time_values[0], False, time_values[0] == values['max_time'], time_values[0] == values['min_time'])} & " + \
        f"{format_value(token_values[0], False, token_values[0] == values['max_tokens'], token_values[0] == values['min_tokens'])} & " + \
        f"{format_value(correctness_values[1], True, correctness_values[1] == values['max_correctness'], correctness_values[1] == values['min_correctness'])} & " + \
        f"{format_value(precision_values[1], True, precision_values[1] == values['max_precision'], precision_values[1] == values['min_precision'])} & " + \
        f"{format_value(time_values[1], False, time_values[1] == values['max_time'], time_values[1] == values['min_time'])} & " + \
        f"{format_value(token_values[1], False, token_values[1] == values['max_tokens'], token_values[1] == values['min_tokens'])} \\\\ \n"

latex_table += r"\midrule" + "\n"
latex_table += "Average & " + \
    f"{format_value(avg_correctness[0], True, avg_correctness[0] == max_avg_correctness, avg_correctness[0] == min_avg_correctness)} & " + \
    f"{format_value(avg_precision[0], True, avg_precision[0] == max_avg_precision, avg_precision[0] == min_avg_precision)} & " + \
    f"{format_value(avg_time[0], False, avg_time[0] == max_avg_time, avg_time[0] == min_avg_time)} & " + \
    f"{format_value(avg_tokens[0], False, avg_tokens[0] == max_avg_tokens, avg_tokens[0] == min_avg_tokens)} & " + \
    f"{format_value(avg_correctness[1], True, avg_correctness[1] == max_avg_correctness, avg_correctness[1] == min_avg_correctness)} & " + \
    f"{format_value(avg_precision[1], True, avg_precision[1] == max_avg_precision, avg_precision[1] == min_avg_precision)} & " + \
    f"{format_value(avg_time[1], False, avg_time[1] == max_avg_time, avg_time[1] == min_avg_time)} & " + \
    f"{format_value(avg_tokens[1], False, avg_tokens[1] == max_avg_tokens, avg_tokens[1] == min_avg_tokens)} \\\\ \n"

latex_table += r"""
\end{tabular}
}

\makebox[\textwidth][c]{
\begin{tabular}{lcccc|cccc}
\toprule
 \textbf{Query type} & \multicolumn{4}{c}{\textbf{Tavily}} & \multicolumn{4}{c}{\textbf{Exa}} \\ \cline{2-9}
 & Correctness & Context Precision & Time (s) & Tokens & Correctness & Context Precision & Time (s) & Tokens \\ \hline
"""

# Generate the bottom table rows
for query in query_types:
    values = get_max_min_values_for_query(query)

    correctness_values = [
        sum(tavily_scores[query]) / len(tavily_scores[query]),
        sum(exa_scores[query]) / len(exa_scores[query])
    ]

    precision_values = [
        sum(tavily_precision[query]) / len(tavily_precision[query]),
        sum(exa_precision[query]) / len(exa_precision[query])
    ]

    time_values = [
        sum(tavily_times[query]) / len(tavily_times[query]),
        sum(exa_times[query]) / len(exa_times[query])
    ]

    token_values = [
        sum(tavily_tokens[query]) / len(tavily_tokens[query]),
        sum(exa_tokens[query]) / len(exa_tokens[query])
    ]

    latex_table += f"{query.capitalize().replace('_', ' ')} & " + \
        f"{format_value(correctness_values[0], True, correctness_values[0] == values['max_correctness'], correctness_values[0] == values['min_correctness'])} & " + \
        f"{format_value(precision_values[0], True, precision_values[0] == values['max_precision'], precision_values[0] == values['min_precision'])} & " + \
        f"{format_value(time_values[0], False, time_values[0] == values['max_time'], time_values[0] == values['min_time'])} & " + \
        f"{format_value(token_values[0], False, token_values[0] == values['max_tokens'], token_values[0] == values['min_tokens'])} & " + \
        f"{format_value(correctness_values[1], True, correctness_values[1] == values['max_correctness'], correctness_values[1] == values['min_correctness'])} & " + \
        f"{format_value(precision_values[1], True, precision_values[1] == values['max_precision'], precision_values[1] == values['min_precision'])} & " + \
        f"{format_value(time_values[1], False, time_values[1] == values['max_time'], time_values[1] == values['min_time'])} & " + \
        f"{format_value(token_values[1], False, token_values[1] == values['max_tokens'], token_values[1] == values['min_tokens'])} \\\\ \n"

latex_table += r"\midrule" + "\n"
latex_table += "Average & " + \
    f"{format_value(avg_correctness[2], True, avg_correctness[2] == max_avg_correctness, avg_correctness[2] == min_avg_correctness)} & " + \
    f"{format_value(avg_precision[2], True, avg_precision[2] == max_avg_precision, avg_precision[2] == min_avg_precision)} & " + \
    f"{format_value(avg_time[2], False, avg_time[2] == max_avg_time, avg_time[2] == min_avg_time)} & " + \
    f"{format_value(avg_tokens[2], False, avg_tokens[2] == max_avg_tokens, avg_tokens[2] == min_avg_tokens)} & " + \
    f"{format_value(avg_correctness[3], True, avg_correctness[3] == max_avg_correctness, avg_correctness[3] == min_avg_correctness)} & " + \
    f"{format_value(avg_precision[3], True, avg_precision[3] == max_avg_precision, avg_precision[3] == min_avg_precision)} & " + \
    f"{format_value(avg_time[3], False, avg_time[3] == max_avg_time, avg_time[3] == min_avg_time)} & " + \
    f"{format_value(avg_tokens[3], False, avg_tokens[3] == max_avg_tokens, avg_tokens[3] == min_avg_tokens)} \\\\ \n"

latex_table += r"""
\bottomrule
\end{tabular}
}
\caption{Average Answer Correctness, Context Precision, Retrieval Time, and Tokens for AskNews, JinaAI, Tavily, and Exa, computed using Ragas across 5 trials for each query type.}
\label{table:average_values}
\end{table}
"""

# Save the LaTeX table to a file
with open('latex_table.tex', 'w') as f:
    f.write(latex_table)

print(latex_table)



\begin{table}[ht]
\centering
\makebox[\textwidth][c]{
\begin{tabular}{lcccc|cccc}
\toprule
 \textbf{Query type} & \multicolumn{4}{c}{\textbf{AskNews}} & \multicolumn{4}{c}{\textbf{JinaAI}} \\ \cline{2-9}
 & Correctness & Context Precision & Time (s) & Tokens & Correctness & Context Precision & Time (s) & Tokens \\ \hline
Latest news 1 & \textcolor{darkgreen}{0.53} & \textcolor{darkgreen}{0.79} & \textcolor{darkgreen}{1.13} & 1312.67 & 0.21 & \textcolor{red}{0.00} & \textcolor{red}{18.54} & \textcolor{darkgreen}{543.00} \\ 
Latest news 2 & \textcolor{darkgreen}{0.35} & 0.51 & 1.15 & 1137.00 & 0.24 & \textcolor{darkgreen}{0.56} & \textcolor{red}{4.83} & \textcolor{darkgreen}{671.00} \\ 
General web 1 & \textcolor{darkgreen}{1.00} & \textcolor{darkgreen}{0.95} & \textcolor{darkgreen}{0.22} & 1317.00 & 0.46 & 0.50 & \textcolor{red}{5.41} & \textcolor{darkgreen}{601.33} \\ 
General web 2 & 0.73 & \textcolor{darkgreen}{0.48} & \textcolor{darkgreen}{0.24} & \textcolor{red}{1473.00} & \textco